In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import print_function, division

## Preambulo

In [14]:
# Verificando que está el dataset
import os
from drums_classifier_utils import prep_dataset
if not os.path.isdir('drums'):
    prep_dataset() 

In [24]:
# Leyendo el dataset
from scipy.io import wavfile
max_duration = 1000  # ms
drum_types = ['crash', 'tom', 'hat', 'snare', 'kick']

all_dirs = [d_dir for d_dir in os.listdir('drums') if os.path.isdir(os.path.join('drums', d_dir))]
all_wavs = [os.path.join('drums', d_dir, wf)
            for d_dir in all_dirs 
            for wf in os.listdir(os.path.join('drums', d_dir)) if wf.endswith('.wav')]

metadata = []
for wf in all_wavs:
    try:
        sr, wav = wavfile.read(wf)
    except Exception as e:
        continue
        
    dur = len(wav) / float(sr) * 1000
    if dur > max_duration:
        continue
        
    category = 'None'
    for dt in drum_types:
        if wf.find(dt) > 0:
            category = dt
    
    metadata.append({
        "filename": wf,
        "category": category,
        "sr": sr,
        "ch": wav.shape[1] if len(wav.shape) > 1 else 1,
        "dur": dur,
    })


In [29]:
global_sr = metadata[0]
classes = drum_types
classes_n = len(classes)

type_translate = dict([(dt, idx) for idx, dt in enumerate(classes)])
def to_one_hot(dt):
    ret = np.zeros(classes_n)
    ret[type_translate[dt]] = 1
    return ret

def to_signal(wf):
    ret = np.zeros(global_sr)
    sig = wavfile.read(wf)[1]
    ret[:len(sig)] = sig
    return ret

In [33]:
# :: [(señal :: [int], clase en one hot :: [00010])]
# Señales de audios todos durando 1 segundo con mismo sample rate.
# No está la clase 'Others' o None. 
signal_one_hot_type = [(to_signal(md['filename']), to_one_hot(md['category']))  
                       for md in metadata
                       if md['category'] != 'None']

{'category': 'None', 'sr': 44100, 'dur': 311.54195011337873, 'ch': 1, 'filename': 'drums/Simmons SD-8/MaxV - SQSM-01.wav'}


TypeError: expected sequence object with len >= 0 or a single integer